[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/028_pytorch_nn/pytorch_nn.ipynb)

# Pytorch - Redes Neuronales

En el post [anterior](https://sensioai.com/blog/027_pytorch_intro) hicimos una introducción al framework de `redes neuronales` `Pytorch`. Hablamos de sus tres elementos fundamentales: el objeto `tensor` (similar al `array` de `NumPy`) `autograd` (que nos permite calcular derivadas de manera automáticas) y el soporte GPU. En este post vamos a entrar en detalle en la  funcionalidad que nos ofrece la librería para diseñar redes neuronales de manera flexible.

In [1]:
import torch

## Modelos secuenciales

La forma más sencilla de definir una `red neuronal` en `Pytorch` es utilizando la clase `Sequentail`. Esta clase nos permite definir una secuencia de capas, que se aplicarán de manera secuencial (las salidas de una capa serán la entrada de la siguiente). Ésto ya lo conocemos de posts anteriores, ya que es la forma ideal de definir un `Perceptrón Multicapa`.

In [30]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# D_in, H1, H2, D_out = 784, 100, 50, 10
# model = torch.nn.Sequential(
#     torch.nn.Linear(D_in, H1),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H1, H2),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H2, D_out),
# )


El modelo anterior es un `MLP` con 784 entradas, 100 neuronas en la capa oculta y 10 salidas. Podemos usar este modelo para hacer un clasificador de imágenes con el dataset MNIST. Pero primero, vamos a ver como podemos calcular las salidas del modelo a partir de unas entradas de ejemplo.

In [3]:
outputs = model(torch.randn(64, 784))
outputs.shape

torch.Size([64, 10])

In [4]:
print(outputs[0][:])

tensor([ 0.1509, -0.0145, -0.0042,  0.0770,  0.1880, -0.1558, -0.4563,  0.0551,
         0.1975, -0.0410], grad_fn=<SliceBackward0>)


Como puedes ver, simplemente le pasamos los inputs al modelo (llamándolo como una función). En este caso, usamos un tensor con 64 vectores de 784 valores. Es importante remarcar que los modelos de `Pytorch` (por lo general) siempre esperan que la primera dimensión sea la dimensión *batch*. Si queremos entrenar esta red en una GPU, es tan sencillo como

In [5]:
model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)

Vamos a ver ahora como entrenar este modelo con el dataset MNIST.

In [6]:
from sklearn.datasets import fetch_openml

# descarga datos

mnist = fetch_openml('mnist_784', version=1)
X, Y = mnist["data"], mnist["target"]

X.shape, Y.shape

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


((70000, 784), (70000,))

In [7]:
import numpy as np

# normalización y split
import numpy as np
x_2=np.array(X)
y_2=np.array(Y)

# normalización y split

X_train =x_2[:60000] / 255.
X_test =x_2[60000:] / 255.
y_train = y_2[:60000].astype(np.int)
y_test = y_2[60000:].astype(np.int)



# X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(np.float32), Y[60000:].astype(np.float32)

<ipython-input-7-a6ea223a5e1a>:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train = y_2[:60000].astype(np.int)
<ipython-input-7-a6ea223a5e1a>:13: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for

In [8]:
# función de pérdida y derivada

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [9]:
# X_train

In [10]:
torch.cuda.is_available()

True

In [11]:
print(X)

       pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
69995     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
69996     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
69997     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
69998     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
69999     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

       pixel10  ...  pixel7

In [12]:
# convertimos datos a tensores y copiamos en gpu

X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

# bucle entrenamiento
epochs = 100
lr = 0.8
log_each = 10
l = []
for e in range(1, epochs + 1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/100 Loss 1.67451
Epoch 20/100 Loss 1.42091
Epoch 30/100 Loss 1.17419
Epoch 40/100 Loss 1.01074
Epoch 50/100 Loss 0.88845
Epoch 60/100 Loss 0.79499
Epoch 70/100 Loss 0.72655
Epoch 80/100 Loss 0.67353
Epoch 90/100 Loss 0.62867
Epoch 100/100 Loss 0.59141


Como puedes observar en el ejemplo, podemos calcular la salida del modelo con una simple línea. Luego calculamos la función de pérdida, y llamando a la función `backward` `Pytorch` se encarga de calcular las derivadas de la misma con respecto a todos los parámetros del modelo automáticamente (si no queremos acumular estos gradientes, nos aseguramos de llamar a la función `zero_grad` para ponerlos a cero antes de calcularlos). Por útlimo, podemos iterar por los parámetros del modelo aplicando la regla de actualización deseada (en este caso usamos `descenso por gradiente`).

In [13]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

0.9307

Existen algunos tipos de capas que se comportan diferente en función de si estamos entrenando la red o usándola para generar predicciones. Podemos controlar el modo en el que queremos que esté nuestra red con las funciones `train` y `eval`.

## Optimizadores y Funciones de pérdida

En el ejemplo anterior hemos calculado la función de pérdida y aplicado la regla de optimización de forma manual. Sin embargo, `Pytorch` nos ofrece funcionalidad que nos abstrae estos cálculos ofreciendo además flexibilidad para aplicar diferentes funciones de pérdida o algoritmos de optimización de manera sencilla. Podemos encontrar diferentes funciones de pérdida ya implementadas en el paquete `torch.nn`.

In [14]:
criterion = torch.nn.CrossEntropyLoss()

Mientras que los optimizadores se encuentran en el paquete `torch.optim`

In [15]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

Puedes ver la lista completa de funciones de pérdida y optimizadores disponibles en la [documentación](https://pytorch.org/docs/stable/index.html), aunque como ya has visto siempre puedes definir los tuyos propios fácilmente.

Una vez definidos estos dos objetos, nuestro bucle de entrenamiento se simplifica considerablemente.

In [16]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.74945
Epoch 20/100 Loss 1.47107
Epoch 30/100 Loss 1.19533
Epoch 40/100 Loss 1.00653
Epoch 50/100 Loss 0.87859
Epoch 60/100 Loss 0.79244
Epoch 70/100 Loss 0.72454
Epoch 80/100 Loss 0.67162
Epoch 90/100 Loss 0.62822
Epoch 100/100 Loss 0.59077


0.931

## Modelos custom

Si bien en muchos casos definir una `red neuronal` como una secuencia de capas es suficiente, en otros casos será un factor limitante. Un ejemplo son las redes residuales, en las que no sólo utilizamos la salida de una capa para alimentar la siguiente si no que, además, le sumamos su propia entrada. Este tipo de arquitectura no puede ser definida con la clase `Sequential`, y para ello necesitamos hacer un modelo *customizado*. Para ello, `Pytroch` nos ofrece la siguiente sintaxis.

In [17]:
# creamos una clase que hereda de `torch.nn.Module`

class ModeloPersonalizado(torch.nn.Module):

    # constructor
    def __init__(self, D_in, H, D_out):

        # llamamos al constructor de la clase madre
        super(ModeloPersonalizado, self).__init__()

        # definimos nuestras capas
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)

    # lógica para calcular las salidas de la red
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

En primer lugar, necesitamos definir una nueva clase que herede de la clase `torch.nn.Module`. Esta clase madre aportará toda la funcionalidad esencial que necesita una `red neuronal` (soporte GPU, iterar por sus parámeteros, etc). Luego, en esta clase necesitamos definir mínimos dos funciones:

- `init`: en el constructor llamaremos al constructor de la clase madre y después definiremos todas las capas que querramos usar en la red.
- `forward`: en esta función definimos toda la lógica que aplicaremos desde que recibimos los inputs hasta que devolvemos los outputs.

En el ejemplo anterior simplemente hemos replicado la misma red (puedes conseguir el mismo efecto usando la clase `Sequential`).

In [18]:
model = ModeloPersonalizado(784, 100, 10)
# Codigo para saber si el modelo esta votando los datos en las cantidades correctas
x_prueba = torch.randn(64, 784)
print(x_prueba)
outputs = model(x_prueba)
outputs.shape

tensor([[-0.2671, -0.7899,  0.7040,  ...,  0.7803, -1.0434, -0.9068],
        [-1.9547, -0.3460, -0.7553,  ..., -1.8140,  0.6800,  1.1549],
        [-1.1048, -0.1703,  0.2703,  ..., -0.7449,  0.0305,  0.3261],
        ...,
        [-1.1914, -0.5317, -0.5634,  ..., -0.3309, -0.1173,  0.2751],
        [-0.2306, -0.0209, -0.5257,  ..., -1.5199, -0.7754,  1.4482],
        [ 0.5666, -0.8594,  0.4785,  ..., -0.1709,  1.4044, -0.9566]])


torch.Size([64, 10])

Ahora, podemos entrenar nuestra red de la misma forma que lo hemos hecho anteriormente.

In [19]:
model.to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.72247
Epoch 20/100 Loss 1.41299
Epoch 30/100 Loss 1.16305
Epoch 40/100 Loss 0.99068
Epoch 50/100 Loss 0.88280
Epoch 60/100 Loss 0.79796
Epoch 70/100 Loss 0.72887
Epoch 80/100 Loss 0.67554
Epoch 90/100 Loss 0.63268
Epoch 100/100 Loss 0.59675


0.924

Aquí puedes ver otro ejemplo de como definir un `MLP` con conexiones residuales, algo que no podemos hacer simplemente usando un modelo secuencial.

In [20]:
class ModelCustom2(torch.nn.Module):

    def __init__(self, D_in, H, D_out):
        super(ModelCustom2, self).__init__()
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        x1 = self.fc1(x)
        x = self.relu(x1)
        x = self.fc2(x + x1)
        return x

In [21]:
model = ModelCustom2(784, 100, 10).to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.59718
Epoch 20/100 Loss 1.16633
Epoch 30/100 Loss 0.95690
Epoch 40/100 Loss 0.85947
Epoch 50/100 Loss 0.77207
Epoch 60/100 Loss 0.71366
Epoch 70/100 Loss 0.66759
Epoch 80/100 Loss 0.62922
Epoch 90/100 Loss 0.59807
Epoch 100/100 Loss 0.57227


0.9089

De esta manera, tenemos mucha flexibilidad para definir nuestras redes.

## Accediendo a las capas de una red

En ocasiones queremos acceder a una capa en particular de nuestra red. Para ello, podemos acceder utilizando su nombre.

In [22]:
model

ModelCustom2(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)

In [23]:
model.fc1

Linear(in_features=784, out_features=100, bias=True)

También podemos acceder directamente a los tensores que contienen los parámetros con las propiedades adecuadas

In [24]:
model.fc1.weight

Parameter containing:
tensor([[ 0.0068,  0.0033,  0.0160,  ..., -0.0331, -0.0101,  0.0156],
        [-0.0008, -0.0148,  0.0189,  ...,  0.0105, -0.0180,  0.0171],
        [-0.0244,  0.0133,  0.0284,  ...,  0.0151,  0.0058,  0.0238],
        ...,
        [-0.0332, -0.0151,  0.0107,  ..., -0.0110, -0.0017, -0.0163],
        [-0.0114,  0.0126, -0.0074,  ...,  0.0022, -0.0123,  0.0131],
        [-0.0238, -0.0274, -0.0096,  ..., -0.0038,  0.0178, -0.0116]],
       device='cuda:0', requires_grad=True)

In [25]:
model.fc1.bias

Parameter containing:
tensor([ 0.0312,  0.0071, -0.0019, -0.0083, -0.0097, -0.0195, -0.0104, -0.0238,
         0.0104,  0.0103, -0.0443,  0.0544,  0.0652,  0.0261, -0.0574,  0.0307,
         0.0369, -0.0345,  0.0578,  0.0358, -0.0369, -0.0544, -0.0045, -0.0144,
        -0.0249,  0.0012,  0.0542,  0.0280,  0.0222,  0.0316, -0.0086,  0.0108,
        -0.0110,  0.0246, -0.0014,  0.0218,  0.0038, -0.0226, -0.0298,  0.0398,
         0.0173, -0.0554,  0.0261, -0.0341, -0.0764,  0.0012,  0.0466, -0.0383,
         0.0703,  0.0237, -0.0285,  0.0186,  0.0341, -0.0072, -0.0179,  0.0351,
        -0.0336,  0.0407,  0.0040, -0.0404,  0.0452, -0.0091,  0.0708, -0.0176,
         0.0677, -0.0057,  0.0032,  0.0215,  0.0639,  0.0470, -0.0100, -0.0122,
         0.0038, -0.0139, -0.0348,  0.0328,  0.0913,  0.0261,  0.0193, -0.0055,
         0.0390,  0.0171, -0.0168,  0.0147, -0.0018, -0.0015,  0.0414,  0.0704,
         0.0272,  0.0076, -0.0040,  0.0691,  0.0634,  0.0003,  0.0551, -0.0037,
         0.0835,  

Es posible sobreescribir una capa de la siguiente manera

In [26]:
model.fc2 = torch.nn.Linear(100, 1)

model

ModelCustom2(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)

Ahora, la capa final de nuestra red tiene solo una salida. Esta nueva capa ha sido inicializada de manera aleatoria, por lo que esta nueva red no nos va a servir de mucho. Sin embargo, podríamos volver a entrenar esta red en otro problema en el que solo necesitemos una salida aprovechando los pesos que ya hemos entrenado anteriormente con el dataset MNIST. Esto es la base del *transfer learning*, una técnica que utilizaremos muchísimo más adelante y la cual explicaremos en detalle.

A continuación encontrarás varios trucos a la hora de crear redes neuronales a partir de otras que te pueden resultar útiles.

In [27]:
# obtener una lista con las capas de una red

list(model.children())

[Linear(in_features=784, out_features=100, bias=True),
 ReLU(),
 Linear(in_features=100, out_features=1, bias=True)]

In [28]:
# crear nueva red a partir de la lista (excluyendo las útlimas dos capa)

new_model = torch.nn.Sequential(*list(model.children())[:-2])
new_model

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
)

In [29]:
# crear nueva red a partir de la lista (excluyendo las útlima capa)

new_model = torch.nn.ModuleList(list(model.children())[:-1])
new_model

ModuleList(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
)

## Resumen

En este post hemos visto la funcionalidad que `Pytorch` nos ofrece a la hora de definir y entrenar nuestras `redes neuronales`. El paquete `torch.nn` contiene todo lo necesario para diseñar nuestros modelos, ya sea de manera secuencial o con una clase *custom* para arquitecturas más complicadas. También nos da muchas funciones de pérdida que podemos usar directamente para entrenar las redes. Te recomiendo encarecidamente que le eches un vistazo a la [documentación](https://pytorch.org/docs/stable/nn.html) par hacerte una idea de todo lo que puedes hacer. También hemos visto como el paquete `torch.optim` nos oferece algoritmos de optimización que también nos hacen la vida más fácil a la hora de entrenar nuestras redes.